In [1]:
import Task_two as two
import bn256
import numpy as np
import random
from PIL import Image


In [2]:
### вектор, який будується з фото використовується в якості регістрації
### вектор для аутентефікації будується рандомно

In [3]:
def reg_vec_image(fotoname):
    img = Image.open(fotoname)
    gray = img.convert('L')
    bw = gray.point(lambda x: x)
    data = np.array(bw.getdata())
    data = np.where(data >= 128, 1, -1)
    return data


In [4]:
v1 = reg_vec_image("eye_for_4 task.jpg")

In [5]:
import functools

def benchmark(f):
    
    import time
    @functools.wraps(f)             
    def _benchmark(*args, **kw):
        t =time.perf_counter()
        rez = f(*args, **kw)
        t = time.perf_counter() - t        
        print('{0} time elapsed {1:.8f} s.'.format(f.__name__, t))
        return rez
    return _benchmark

In [6]:
len_v1_or_v2 = len(v1)

In [7]:
v2 = np.random.choice([1, -1], len_v1_or_v2)


In [8]:
v1, v2

(array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1,  1,  1,  1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1, -1, -1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1,  1,  1, -1, -1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1,  1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1,  1,  1,  1,  1, -1,
        -1, -1, -1, -1, -1,  1,  1,  1, -1,  1, -1, -1, -1,  1,  1,  1, -1,
        -1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  1,  1,
         1, -1, -1,  1,  1,  1, -1, -1,  1,  1,  1,  1,  1, -1, -1, -1, -1,
        -1, -1,  1,  1,  1,  1,  1,  1, -1, -1,  1,  1,  1, -1, -1,  1,  1,
         1, 

In [9]:
hm = two.hamming_distance(v1, v2)

In [10]:
G1 = bn256.curve_G
G2 = bn256.twist_G

In [11]:
v1points = [G2.scalar_mul(bn256.order + i) for i in v1]
v2points  = [G1.scalar_mul(bn256.order + i) for i in v2]

In [12]:
def building_keys():
    s = [random.randrange(2, bn256.order) for _ in range(len_v1_or_v2)]
    t = [random.randrange(2, bn256.order) for _ in range(len_v1_or_v2)]
    u = [random.randrange(2, bn256.order) for _ in range(len_v1_or_v2 + 2)]
    v = [random.randrange(2, bn256.order) for _ in range(len_v1_or_v2 + 2)]
    h1 = random.randrange (2, bn256.order)
    h2 = random.randrange (2, bn256.order)
    H1 = G1.scalar_mul(h1)
    H2 = G2.scalar_mul(h2)
    gen1_h = [G1.scalar_mul(int(s[i])).add(H1.scalar_mul(int(t[i]))) for i in range(len(s))]
    gen2_h = [G2.scalar_mul(int(u[i])).add(H2.scalar_mul(int(v[i]))) for i in range(len(u))]
    return [[G2, H2, s, t, gen2_h], [G1, H1, u, v, gen1_h]]
    

In [13]:
msk = building_keys()

In [15]:
@benchmark    
def IPE_Registration(msk, v1points):
    
    r0 = random.randrange(2, bn256.order)
    
    reg_template = []
    reg_template.append(msk[0].scalar_mul(r0))
    reg_template.append(msk[1].scalar_mul(r0))
    for i in range(len_v1_or_v2 + 2):
        reg_template.append(msk[4][i].scalar_mul(r0))
         
    for i in range(len_v1_or_v2):
        reg_template[2] = reg_template[2].add(v1points[i].scalar_mul(msk[2][i]).scalar_mul(bn256.order - 1))
        reg_template[3] = reg_template[3].add(v1points[i].scalar_mul(msk[3][i]).scalar_mul(bn256.order - 1))
        reg_template[i+4] = reg_template[i+4].add(v1points[i])
    
    
    return reg_template

    

In [16]:
result_reg = IPE_Registration(msk[0], v1points)

IPE_Registration time elapsed 229.13250164 s.


In [17]:
@benchmark    
def IPE_Authentication(msk, v2points ):
    r0 = random.randrange(2, bn256.order)
#     print(r0)
    auth_template = []
    for i in range(2):
        auth_template.append(msk[0].scalar_mul(0))
    auth_template.append(msk[0].scalar_mul(r0))
    auth_template.append(msk[1].scalar_mul(r0))
    for i in range(len_v1_or_v2):
        auth_template.append(msk[4][i].scalar_mul(r0).add(v2points[i]))
    for i in range(len_v1_or_v2 + 2):
        auth_template[0] = auth_template[0].add((auth_template[i+2].scalar_mul(msk[2][i])).scalar_mul(bn256.order - 1))
        auth_template[1] = auth_template[1].add(auth_template[i+2].scalar_mul(msk[3][i]).scalar_mul(bn256.order - 1))
    return auth_template

In [18]:
result_aut = IPE_Authentication(msk[1], v2points )

IPE_Authentication time elapsed 57.73551972 s.


In [19]:
@benchmark    
def building_op_ate_e():
    e = bn256.optimal_ate(result_reg[0], result_aut[0])
    for i in range(1, len_v1_or_v2 + 4):
        e = e.mul(bn256.optimal_ate(result_reg[i], result_aut[i]))
    return e
    

In [20]:
e = building_op_ate_e()

building_op_ate_e time elapsed 133.75367593 s.


In [21]:
e

(((48034000875468529081057204324599093253537356572950478156746632214482025429868,53085525718919287722593114683147824740875646157084388770684319903910806768680),(18798944745918306305634753732831929365435230578694775046531608149088584500734,62126817285076125971520308703141224394447512705736539070635740581618486467874),(47622099726246894805875336261643886153462180646224395925705463707753364500315,9594415641529309604543387784652144065972097014018229345792580434615651768892)),((4474353317627756812845845131389175367127535918145379524907475651886153658899,38129357329093419019879468496457502110299387139281050357979537780866722842463),(10488638252263044780125494102679502025940481973351334076138584674936854335184,38881740341596777832283796127796190275090700732091805771072329437780482052101),(43338787592769049253517893796655182000280095163665925062356424335217977075811,47279284881066482683709997358374103829548126347987421851988369941001056211979)))

In [22]:
@benchmark    
def building_logtable():
    spar_one = bn256.optimal_ate(G2, G1)
    log_table = [spar_one.exp(i) for i in range(len_v1_or_v2, -1, -2)]
    spar_two = bn256.optimal_ate(G2, G1.scalar_mul(bn256.order - 1))
    log_table = log_table + [spar_two.exp(i) for i in range(2, len_v1_or_v2 + 1, 2)]
    return log_table

In [23]:
log_table = building_logtable()

building_logtable time elapsed 4.23092946 s.


In [24]:
index_e_log = building_logtable().index(e)

building_logtable time elapsed 4.30304727 s.


In [25]:
log_table.index(e)

210

In [26]:
hm == index_e_log

True